In [1]:
import numpy as np
import ee
import geopandas as gpd
import sys
sys.path.append('../python')
import lem
import rasterio as rio
import os
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
octo_points = gpd.read_file('/Users/gr_1/Downloads/Archive/crn_int_outlets_071625')
octo_basins = gpd.read_file('/Users/gr_1/Downloads/Archive/crn_int_basins_071625')
octo_basins_orig = octo_basins.copy()
octo_points.index = octo_points.OBSID1
octo_basins.index = octo_basins.OBSID1
octo_points = octo_points.loc[octo_basins.index] #Because we can only download all the points at once...
octo_points = octo_points.to_crs('EPSG:4326')
octo_basins = octo_basins.to_crs('EPSG:4326')
octo_basins.geometry = octo_basins.geometry.buffer(octo_basins.area*.05)
octo_points.index = range(len(octo_points))
octo_basins.index = range(len(octo_basins))

/var/folders/5v/804c95h92bx2zr8rfpwyr_yc0000gn/T/ipykernel_8211/2005517598.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  octo_basins.geometry = octo_basins.geometry.buffer(octo_basins.area*.05)
/var/folders/5v/804c95h92bx2zr8rfpwyr_yc0000gn/T/ipykernel_8211/2005517598.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  octo_basins.geometry = octo_basins.geometry.buffer(octo_basins.area*.05)
/Users/gr_1/miniconda3/lib/python3.9/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


In [3]:
octo_basins.geometry[octo_basins.area>15]=None

/var/folders/5v/804c95h92bx2zr8rfpwyr_yc0000gn/T/ipykernel_1160/2453851077.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  octo_basins.geometry[octo_basins.area>15]=None


In [3]:
octo_basins=gpd.read_file('octo_saved_tmp')

In [18]:
# df2 = octo_points.reindex(octo_basins.index).reset_index
# octo_points.iloc[101]
octo_points.index = np.arange(len(octo_points))
octo_points.to_file('octo_points')

#octo_basins.iloc[101]

## Download Octopus data from octopusdata.org
## You can clip the tiles for DEMs using EarthEngine

In [5]:
ee.Initialize()

In [6]:
import json 
from ee.geometry import Geometry
basin_slopes = gpd.GeoDataFrame()
basin_slopes_t = gpd.GeoDataFrame()

shps = []
image = ee.Image("WWF/HydroSHEDS/03VFDEM").setDefaultProjection('EPSG:4326',None,30)
for i,row in octo_basins.iloc[:].iterrows():
    #if i in unique:# remove unique line for full basins
        if row.geometry is None:
            shp = []
        elif row.geometry.type=='MultiPolygon':
                asi = [geom.area for geom in row.geometry.geoms]
                xy2 = row.geometry.geoms[np.argmax(asi)].exterior.xy
                sh = list(zip(xy2[0],xy2[1]))
                shp = Geometry.Polygon(sh)
                print(i)
        else:
            try:
                sh = list(zip(row.geometry[-1].exterior.coords.xy[0],row.geometry[-1].exterior.coords.xy[1]))
            except:
                sh = list(zip(row.geometry.exterior.coords.xy[0],row.geometry.exterior.coords.xy[1]))

            shp = Geometry.Polygon(sh)

        shps.append(shp)
for c in octo_basins.index[3000:]:
    im = image.clip(shps[c])
    mytask = ee.batch.Export.image.toDrive(image=im, folder='data_earth_engine', description='nz_bas_hydroshed_'+ str(c)+'_')
    mytask.start()


/var/folders/5v/804c95h92bx2zr8rfpwyr_yc0000gn/T/ipykernel_1160/1243929309.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif row.geometry.type=='MultiPolygon':


KeyboardInterrupt: 

In [35]:
octo_basins.iloc[16]

OBSID1                                               S086WTS004
OBSID2                                                   NEP065
STUDYID                                                    S086
CRN_SUBCMP                                                  INT
IGSNID                                                       NA
                                    ...                        
EALL_EXT                                               -9999.99
EALLNS                                                 -9999.99
EALLNS_INT                                             -9999.99
EALLNS_EXT                                             -9999.99
geometry      POLYGON ((83.77521144678765 28.9048628529246, ...
Name: S086WTS004, Length: 94, dtype: object

In [27]:
octo_basins.index

Index(['S141WTS002', 'S075WTS009', 'S304WTS005', 'S293WTS010', 'S101WTS021',
       'S088WTS004', 'S045WTS011', 'S143WTS027', 'S319WTS009', 'S094WTS010',
       ...
       'S357WTS022', 'S326WTS016', 'S046WTS061', 'S001WTS001', 'S166WTS018',
       'S006WTS009', 'S180WTS001', 'S326WTS025', 'S166WTS003', 'S326WTS026'],
      dtype='object', name='OBSID1', length=5376)

In [19]:
octo_points

,OBSID1,OBSID2,STUDYID,geometry
0,S141WTS002,Yan,S141,POINT (8712868.525 3907998.274)
1,S075WTS009,WR5,S075,POINT (-13447628.340 6068569.409)
2,S304WTS005,DB05,S304,POINT (805304.785 5732412.080)
3,S293WTS010,T10,S293,POINT (10724492.864 3454661.990)
4,S101WTS021,Nurla,S101,POINT (8570096.683 4069453.027)
...,...,...,...,...
5371,S006WTS009,NY-20A1,S006,POINT (3889943.411 3450748.091)
5372,S180WTS001,Trib 1,S180,POINT (-13759715.818 5490998.947)
5373,S326WTS025,Tnk-25,S326,POINT (15140555.604 4154008.859)
5374,S166WTS003,JPIIDA41,S166,POINT (15335517.986 4257651.677)


In [2]:
def snap_pourpoint(acc,basin_y,basin_x,pt_y,pt_x,target_area, dA):
        x = np.argmin((basin_x - pt_x) ** 2)
        y = np.argmin((basin_y - pt_y) ** 2)
        A1=0
        w=1
        m,n = np.shape(acc)
        print(target_area)
        while A1< target_area/1.2:
                ny, nx = np.where(acc[y-w:y+w+1,x-w:x+w+1]>= np.max(acc[y-w:y+w+1,x-w:x+w+1])/1.25) # We find the closest point within half the max drainage area in the box
                print(w)
                print(A1)
                print(target_area)
                xysnap = np.argmin((ny - w) ** 2 + (nx - w) ** 2)
                ysnap = ny[xysnap] + y - w
                xsnap = nx[xysnap] + x - w
                A1 = acc[ysnap,xsnap]*dA

                w+=1
                if w>.02*(m/2+n/2):
                        break



        # plt.imshow(acc,vmax=1000,vmin=0)
        # plt.plot(x,y,'.')
        # plt.plot(xsnap,ysnap,'.r')
        return ysnap, xsnap



In [3]:
dir = '../example_data/nz_dems/'
basenm = "nz_bas_hydroshed_{}_.tif"
files = os.listdir(dir)


for ptn in range(27):
    DEM = lem.simple_model() #Initiate the DEM object
    DEM.turn_on_off_dynamic_bc('False')#This means that outlets are assigned, not dynamic i.e. everywhere below sea level.
    f = rio.open(dir + basenm.format(str(ptn))) #File containing DEM
    pt = octo_points.loc[ptn] #Find the relevant pour point
    Z=np.float64(np.squeeze(f.read()))
    Z[Z<=0] = np.nan

    m,n = np.shape(Z)
    F.set_z(Z)
    BCX = np.zeros(np.shape(Z))
    
    F.sinkfill()
    F.slp()
    lat = np.array([f.xy(i,0)[1] for i in range(m)])
    lon = np.array([f.xy(0,i)[0] for i in range(n)])
    dL =  111000 * np.cos(np.mean(lat)*np.pi/180) # m per degree
    dx = np.mean(np.diff(lon)) * dL #m per pixel, avg x
    dy = np.mean(np.diff(lat)) * 111000 # m per pixel, avg y
    
    dA = np.abs(dx*dy)
    print(dx)

    F.stack()
    F.acc()
    plt.imshow(F.A,vmax=100)
    acc = F.A.copy()
    L = 111000 # km per deg
    target_area = octo_basins_orig.area[ptn]*dL*L
    ys,xs = snap_pourpoint(acc, lat, lon, pt.geometry.xy[1], pt.geometry.xy[0], target_area, dA)
    F.pour_point = np.int64([ys, xs])
    F.stack()
    F.acc()
    plt.figure()
    plt.imshow(F.A,vmin=0,vmax=2)
    Z = np.zeros(np.shape(F.get_z())).ravel(order='F')
    Zi = F.get_z().ravel(order='F')
    for i in range(len(F.stackij)):
        Z[F.stackij[i]] = Zi[F.stackij[i]]
        
    Z = Z.reshape(m,n,order='F')
    profile = f.profile

    Z[Z==0] = -9999
    with rio.open(dir+'nz_clipped_hydrosheds_{}.tif'.format(str(ptn)), 'w', **profile) as dst:
        dst.write(Z.astype(rio.float64), 1)
    
#plt.imshow(Z)

NameError: name 'octo_points' is not defined